# Sentiment Classification


## Loading the dataset

In [1]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

Using TensorFlow backend.


17465344/17464789 [==============================] - 11s 1us/step


In [3]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

## Train test split

In [4]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [14]:
print(x_train.shape, y_train.shape,  x_test.shape, y_test.shape)


(25000, 300) (25000,) (25000, 300) (25000,)


In [15]:
print("Max val: ", max([max(sequence) for sequence in x_train]))
print("Max len: ", max([len(sequence) for sequence in x_train]))

Max val:  9999
Max len:  300


## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [5]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import os
# create the model
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=maxlen))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 32)           320000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               2400250   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 2,720,501
Trainable params: 2,720,501
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.81%


In [9]:
# save the model to file
model.save('SeqNLP_Project1_imdb_model.h5')
!ls -lrt SeqNLP_Project1_imdb_model.h5

-rw-r--r--  1 kumanish  staff  32672400 Dec 24 12:45 SeqNLP_Project1_imdb_model.h5


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [10]:
model.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.0174 - acc: 0.9938
Epoch 2/10
25000/25000 [==============================] - 27s 1ms/step - loss: 0.0055 - acc: 0.9985
Epoch 3/10
25000/25000 [==============================] - 30s 1ms/step - loss: 0.0014 - acc: 0.9996
Epoch 4/10
25000/25000 [==============================] - 38s 2ms/step - loss: 0.0018 - acc: 0.9993
Epoch 5/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.0238 - acc: 0.9915
Epoch 6/10
25000/25000 [==============================] - 36s 1ms/step - loss: 0.0092 - acc: 0.9965
Epoch 7/10
25000/25000 [==============================] - 34s 1ms/step - loss: 5.7605e-04 - acc: 0.9999
Epoch 8/10
25000/25000 [==============================] - 38s 2ms/step - loss: 2.8175e-05 - acc: 1.0000
Epoch 9/10
25000/25000 [==============================] - 37s 1ms/step - loss: 1.3701e-05 - acc: 1.0000
Epoch 10/10
25000/25000 [==============================] - 36s 1ms/step - loss: 8.8334e-

In [12]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 2s 72us/step
Test score: 1.1443684395879508
Test accuracy: 0.84776
